In [2]:
import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

seed = 8801
np.random.seed(seed)
tf.random.set_seed(seed)

In [24]:
blacklisted_paths = [
  '.DS_Store'
]

In [25]:
commands = np.array(os.listdir('../recordings'))
commands = commands[~np.isin(commands, blacklisted_paths)]
print(sorted(commands))

['hey-player', 'mute', 'next', 'pause', 'play', 'previous', 'rewind', 'unmute']


In [35]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
  directory='../recordings',
  seed=seed
)

Found 160 files belonging to 8 classes.


/Users/paul/.miniforge3/envs/asr/lib/python3.10/multiprocessing/resource_tracker.py:104: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '


In [ ]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
  directory='../recordings',
  batch_size=1,
  validation_split=0.2,
  seed=seed,
  output_sequence_length=16000,
  subset='both',
)

Found 160 files belonging to 8 classes.
Using 128 files for training.
Using 32 files for validation.


/Users/paul/.miniforge3/envs/asr/lib/python3.10/multiprocessing/resource_tracker.py:104: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '


In [27]:
label_names = np.array(train_ds.class_names)
print(sorted(label_names))

['hey-player', 'mute', 'next', 'pause', 'play', 'previous', 'rewind', 'unmute']


In [28]:
train_ds.element_spec

(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [34]:
for audio, _ in train_ds.take(1):
  print(audio.shape)

2022-11-09 19:12:46.621703: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:12:46.621834: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:12:46.621871: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:12:46.621902: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:12:46.622409: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:12:46.622477: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_w

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Data too short when trying to read string
	 [[{{node DecodeWav}}]] [Op:IteratorGetNext]

In [29]:
def squeeze(audio, labels):
  return tf.squeeze(audio, axis=-1), labels

In [30]:
train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [31]:
for example_audio, example_labels in train_ds.take(1):
  print('Audio shape: ', example_audio.numpy().shape)
  print('Audio label: ', example_label.numpy().shape)

2022-11-09 19:11:36.487200: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:11:36.487284: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:11:36.487310: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:11:36.487392: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:11:36.487434: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_wav_op.cc:55 : INVALID_ARGUMENT: Data too short when trying to read string
2022-11-09 19:11:36.487515: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at decode_w

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Data too short when trying to read string
	 [[{{node DecodeWav}}]] [Op:IteratorGetNext]